<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_SEED-ALLDATASET-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input, LeakyReLU
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
from math import sqrt

# ====================================================
# Load data
# ====================================================

##MetroPM
y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')
x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2.npy')

##AQ
y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data-true-window2.npy')
x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data2.npy')

##ELECTRICITY
x=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data2.npy')
y=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data-true-window2.npy')

from sklearn.preprocessing import PowerTransformer
transformer = StandardScaler()
y_transformed = transformer.fit_transform(y.reshape(-1,1)).flatten()
x_train, x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.1, random_state = 42)

# ======================= Model Builder ==========================
def build_model(input_dim):
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=32)); model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=16)); model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=8));  model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(units=1))
    optim = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1)
    model.compile(loss='mean_squared_error', optimizer=optim)
    return model

# ======================= Robustness Runs ========================
seeds = [42, 123, 999, 2024, 7]
rmse_scores = []

for seed in seeds:
    print(f"\n===== Running seed {seed} =====")

    # Set seeds for reproducibility
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Train-test split
    x_train, x_test, y_train, y_test = train_test_split(
        x, y_transformed, test_size=0.1, random_state=seed
    )

    # Build and train
    model = build_model(x.shape[1])

    cp_callback = ModelCheckpoint(
        filepath=f"/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/ckp_seed{seed}.weights.h5",
        monitor='loss', save_best_only=True, save_weights_only=True
    )
    es = EarlyStopping(patience=20, verbose=1, min_delta=1e-4, monitor='loss',
                       mode='min', restore_best_weights=True)

    history = model.fit(
        x_train, y_train,
        epochs=50, batch_size=50,
        validation_split=0.2,
        callbacks=[cp_callback, es],
        verbose=1
    )

    # Evaluation
    y_pred = model.predict(x_test)
    y_test_inv = transformer.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_inv = transformer.inverse_transform(y_pred).flatten()
    rmse = math.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    rmse_scores.append(rmse)

    print(f"Seed {seed} RMSE: {rmse:.4f}")

# ======================= Summary ================================
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
print("\n=== Robustness Report (GDRNet MLP) ===")
print(f"Seeds tested: {seeds}")
print(f"RMSE mean ± std: {mean_rmse:.4f} ± {std_rmse:.4f}")



===== Running seed 42 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.8888 - val_loss: 0.8222
Epoch 2/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.8207 - val_loss: 0.7942
Epoch 3/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7942 - val_loss: 0.7805
Epoch 4/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7825 - val_loss: 0.7674
Epoch 5/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7747 - val_loss: 0.7759
Epoch 6/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7698 - val_loss: 0.7635
Epoch 7/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7658 - val_loss: 0.7602
Epoch 8/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7624 - val_loss: 0.7623
Epoch 9/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 0.7601 - val_loss: 0.7563
Epoch 10/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7579 - val_loss: 0.7586
Epoch 11/100
5040/5040 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7557 - val_loss: 0.7523
Epoch 12

NameError: name 'scaler' is not defined